In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
import together

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')
print(f"using TOGETHER_API_KEY ending in {TOGETHER_API_KEY[-4:]}")

together.api_key = TOGETHER_API_KEY 

print(together.version)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
using TOGETHER_API_KEY ending in c04c
1.0.0


In [3]:
client = together.Together(
  api_key=TOGETHER_API_KEY
)

# Chat Completions
response = client.chat.completions.create(
    model="togethercomputer/llama-2-7b-chat", 
    max_tokens=10, 
    messages=[
        {"role": "user", "content": "hello there"}
    ]
)
print(response.choices[0].message.content)

  Hello! It's nice to meet you


In [12]:
# Completions
response = client.completions.create(
    model="togethercomputer/llama-2-7b", 
    max_tokens=10, 
    prompt="hello there"
)
print(response.choices[0].text)

, I'm a newbie to the forum


In [16]:
# Embeddings
response = client.embeddings.create(model="bert-base-uncased", input=["dog", "puppy"])
print(response.data[0].embedding[:18])

[0.10394287, 0.16479492, -0.24951172, -0.025772095, -0.13427734, -0.026519775, 0.3930664, -0.2052002, -0.043304443, -0.10217285, 0.091796875, -0.041046143, -0.016296387, 0.1496582, -0.37646484, -0.21679688, -0.1842041, 0.23657227]


In [17]:
os.listdir(".")

['python_model_ready.ipynb',
 'quick_start_v1.ipynb',
 'check_new_models.ipynb',
 'antihallucination.jsonl',
 '01_together_python_v1.0.0.ipynb',
 'finetune_inputs.ipynb',
 'together-finetune.ipynb',
 'fastchat.ipynb',
 'antihallucination_dataset.jsonl',
 '500_error_handling.ipynb',
 'async-client.ipynb',
 'together-poe-changes.ipynb',
 'poker_analysis.jsonl',
 'chat_api_v1.ipynb',
 'files.ipynb',
 'correctsample.jsonl',
 'together-start-stop.ipynb',
 'llama2-5xx-v2.ipynb',
 '.ipynb_checkpoints',
 'inference_startup.ipynb']

In [20]:
import json

# Upload Training File
response = client.files.upload("antihallucination_dataset.jsonl")
print(json.dumps(response.model_dump(), indent=4))

{
    "id": "file-4bbd2608-053e-4078-bc23-5af97df99134",
    "object": "file",
    "created_at": 1710361599,
    "purpose": "fine-tune",
    "filename": "antihallucination_dataset.jsonl",
    "bytes": 0,
    "line_count": 0,
    "processed": false
}


In [22]:
# Fine Tuning
response = client.fine_tuning.create(
    training_file="file-4bbd2608-053e-4078-bc23-5af97df99134", 
    model="togethercomputer/llama-2-7b"
)

finetune_id = response.id
print(json.dumps(response.model_dump(), indent=4))

{
    "id": "ft-81309c6d-6d12-46e1-897c-53a16c0c9f8c",
    "training_file": "file-4bbd2608-053e-4078-bc23-5af97df99134",
    "validation_file": "",
    "model": "togethercomputer/llama-2-7b",
    "output_name": "carson/llama-2-7b-2024-03-13-20-39-39",
    "n_epochs": 1,
    "n_checkpoints": 1,
    "batch_size": 32,
    "learning_rate": 1e-05,
    "eval_steps": 0,
    "lora": false,
    "lora_r": 8,
    "lora_alpha": 8,
    "lora_dropout": 0,
    "created_at": "2024-03-13T20:39:39.836Z",
    "updated_at": "2024-03-13T20:39:39.836Z",
    "status": "pending",
    "job_id": "",
    "events": [
        {
            "object": "fine-tune-event",
            "created_at": "2024-03-13T20:39:39.836Z",
            "level": "",
            "message": "Fine tune request created",
            "type": "JOB_PENDING",
            "param_count": 0,
            "token_count": 0,
            "wandb_url": "",
            "hash": "",
            "checkpoint_path": "",
            "model_path": "",
        

In [39]:
finetune_response = client.fine_tuning.retrieve(id=finetune_id)

print(finetune_response.status)

FinetuneJobStatus.STATUS_PENDING


# Async

In [40]:
```python

async_client = together.AsyncTogether(
  api_key=TOGETHER_API_KEY
)

# Chat Completions
response = await async_client.chat.completions.create(
    model="togethercomputer/llama-2-7b-chat", 
    max_tokens=10, 
    messages=[
        {"role": "user", "content": "hello there"}
    ]
)
print(response.choices[0].message.content)

```

modify your answer above with the information above that this client is already asynchronous

  Hello! It's nice to meet you


# Comaprison

In [53]:
import os
import time
from together import Together

TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')

def sync_chat_completion(messages, max_tokens):
    client = Together(api_key=TOGETHER_API_KEY)
    
    start_time = time.time()
    
    for message in messages:
        response = client.chat.completions.create(
            model="togethercomputer/llama-2-7b-chat", 
            max_tokens=max_tokens, 
            messages=[{"role": "user", "content": message}]
        )
        print(response.choices[0].message.content)
    
    end_time = time.time()
    print("Synchronous total execution time:", end_time - start_time, "seconds")

In [56]:
import os
import asyncio
from together import AsyncTogether

TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')

async def async_chat_completion(messages, max_tokens):
    async_client = AsyncTogether(api_key=TOGETHER_API_KEY)
    
    start_time = time.time()
    
    tasks = [async_client.chat.completions.create(
                model="togethercomputer/llama-2-7b-chat", 
                max_tokens=max_tokens, 
                messages=[{"role": "user", "content": message}]
             ) for message in messages]
             
    responses = await asyncio.gather(*tasks)
    
    for response in responses:
        print(response.choices[0].message.content)
    
    end_time = time.time()
    print("Asynchronous total execution time:", end_time - start_time, "seconds")

def run_async_example(messages):
    asyncio.run(async_chat_completion(messages, 32))

In [55]:
messages = ["hi there what is the meaning of life?", "What country is Paris in?"]
sync_chat_completion(messages, 32)
await async_chat_completion(messages, 32)

  The meaning of life is a question that has puzzled philosophers, theologians, and scientists for centuries. There are many different perspectives
  Paris is located in France. It is the capital and largest city of France, situated in the northern central part of the country.
Synchronous total execution time: 0.7738921642303467 seconds
  The meaning of life is a question that has puzzled philosophers, theologians, and scientists for centuries. There are many different perspectives
  Paris is located in France. It is the capital and largest city of France, situated in the northern central part of the country.
Asynchronous total execution time: 0.4429478645324707 seconds
